In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import qutip

# ------------------------
# Configuration
# ------------------------
#use_heterogeneous_coupling = False  # If True, use heterogeneous hyperfine coupling
#use_heterogeneous_distribution = "normal"
n_nuclei = 2

# ------------------------
# Parameter
# ------------------------
MHz = 1e6
omega_eZ = 0 #2 * np.pi * 50 * MHz # Electron Zeeman frequency, 50 MHz
omega_NZ = 0 #2 * np.pi * 5 * MHz  # Nuclear Zeeman frequency, 5 MHz
a_j = 2 * np.pi * 0.5 * MHz # Hyperfine coupling constant, 0.5 MHz

# ------------------------
# Operator
# ------------------------
sx, sy, sz = 0.5 * qutip.sigmax(), 0.5 * qutip.sigmay(), 0.5 * qutip.sigmaz()
sp, sm = 0.5 * qutip.sigmap(), 0.5 * qutip.sigmam()

# electron operator
def e(op):
    ops = [op] + [qutip.qeye(2) for _ in range(n_nuclei)]
    return qutip.tensor(ops)

# nuclear operator k
def n(op, k):
    ops = [qutip.qeye(2) for _ in range(n_nuclei + 1)]
    ops[k+1] = op
    return qutip.tensor(ops)

# ------------------------
# Hamiltonian
# ------------------------
H_eZ = omega_eZ * e(sz)
H_NZ = omega_NZ * sum(n(sz, k) for k in range(n_nuclei))
# TODO: Move a_j inside the sum for heterogeneous coupling
H_hf = a_j * sum(e(sp) * n(sm,k) + e(sm) * n(sp,k) + e(sz) * n(sz,k) for k in range(n_nuclei))

H_esr = 0 # TODO: Add Electron spin resonance term (not used here)
H_nmr = 0 # TODO: Add Nuclear magnetic resonance term (not used here)

H = H_eZ + H_NZ + H_hf + H_esr + H_nmr

# ------------------------
# Initial State
# ------------------------
up, down = qutip.basis(2,0), qutip.basis(2,1)
psi_e0 = up
psi_n0 = qutip.tensor([down for _ in range(n_nuclei)])
psi0 = qutip.tensor(psi_e0, psi_n0)

# ------------------------
# Time Evolution
# ------------------------
tgrid = np.linspace(0, 10e-6, 2000)  # Time grid from 0 to 10 us, 2000 points, i.e. dt = 5 ns
P_up_op = e(up * up.dag())
P_n_up_ops = [n(up * up.dag(), k) for k in range(n_nuclei)]
# Electron spin operators
S_ops = [e(sx), e(sy), e(sz)]

result = qutip.sesolve(H, psi0, tgrid, e_ops=[P_up_op] + P_n_up_ops + S_ops)

P_e_up_t = result.expect[0]
P_n_up_t = result.expect[1:1+n_nuclei]
Sx_t, Sy_t, Sz_t = result.expect[1+n_nuclei:]


# ------------------------
# Plot
# ------------------------
plt.figure(figsize=(7,4))
plt.plot(tgrid*1e6, P_e_up_t, label="Electron")
for k in range(n_nuclei):
    plt.plot(tgrid*1e6, P_n_up_t[k], label=f"Nucleus {k}")
plt.xlabel("time [µs]")
plt.ylabel(r"$P_\uparrow(t)$")
plt.title("Electron–nuclear flip-flop probabilities")
plt.legend()
plt.tight_layout()
plt.show()

# ------------------------
# Plot electron spin components
# ------------------------
plt.figure(figsize=(7,4))
plt.plot(tgrid*1e6, Sx_t, label=r"<Sx>")
plt.plot(tgrid*1e6, Sy_t, label=r"<Sy>")
plt.plot(tgrid*1e6, Sz_t, label=r"<Sz>")
plt.xlabel("time [µs]")
plt.ylabel("Expectation values")
plt.title("Electron spin components")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import qutip

# ------------------------
# Configuration
# ------------------------
use_heterogeneous_coupling = True
hetero_dist = "uniform"   # "uniform" or "normal"
n_nuclei = 2

# ------------------------
# Parameters
# ------------------------
MHz = 1e6
omega_eZ = 0
omega_NZ = 0
a_mean = 2 * np.pi * 0.5 * MHz

# ESR parameters
omega_1_MW = 2*np.pi*0.05*MHz
omega_MW = 2*np.pi*5*MHz

# NMR parameters
omega_1_RF = 2*np.pi*0.02*MHz
omega_RF = 2*np.pi*1*MHz

# ------------------------
# Spin operators
# ------------------------
sx, sy, sz = 0.5*qutip.sigmax(), 0.5*qutip.sigmay(), 0.5*qutip.sigmaz()
sp, sm = 0.5*qutip.sigmap(), 0.5*qutip.sigmam()

# ------------------------
# Operators in full space
# ------------------------
def e(op):
    return qutip.tensor([op] + [qutip.qeye(2) for _ in range(n_nuclei)])

def n(op, k):
    ops = [qutip.qeye(2)]*(1+n_nuclei)
    ops[k+1] = op
    return qutip.tensor(ops)

# ------------------------
# Hyperfine couplings
# ------------------------
if use_heterogeneous_coupling:
    if hetero_dist == "uniform":
        a_list = a_mean * np.random.uniform(0.5, 1.5, n_nuclei)
    else:
        a_list = np.random.normal(a_mean, 0.2*a_mean, n_nuclei)
else:
    a_list = [a_mean]*n_nuclei

# ------------------------
# Hamiltonian: static terms
# ------------------------
H_eZ = omega_eZ * e(sz)
H_NZ = omega_NZ * sum(n(sz, k) for k in range(n_nuclei))
H_hf = sum(a_list[k] * (e(sp)*n(sm,k) + e(sm)*n(sp,k) + e(sz)*n(sz,k))
          for k in range(n_nuclei))

# ------------------------
# ESR (time-dependent)
# ------------------------
H_esr_x = e(sx)
H_esr_y = e(sy)
H_esr = [[omega_1_MW * H_esr_x, "cos(wmw*t)"],
         [-omega_1_MW * H_esr_y, "sin(wmw*t)"]]

# ------------------------
# NMR (time-dependent)
# ------------------------
H_nmr = []
for k in range(n_nuclei):
    H_nmr.append([omega_1_RF * n(sx, k), "cos(wrf*t)"])

# ------------------------
# Full Hamiltonian list
# ------------------------
H = [H_eZ + H_NZ + H_hf] + H_esr + H_nmr

# ------------------------
# Initial state
# ------------------------
up, down = qutip.basis(2,0), qutip.basis(2,1)
psi0 = qutip.tensor(up, qutip.tensor([down]*n_nuclei))

# ------------------------
# Time evolution
# ------------------------
tgrid = np.linspace(0, 10e-6, 2000)

args = {"wmw": omega_MW, "wrf": omega_RF}

P_up_e = e(up*up.dag())
P_up_n = [n(up*up.dag(), k) for k in range(n_nuclei)]
S_ops = [e(sx), e(sy), e(sz)]

result = qutip.sesolve(H, psi0, tgrid,
                       e_ops=[P_up_e] + P_up_n + S_ops,
                       args=args)

# ------------------------
# Extract results
# ------------------------
P_e_up_t = result.expect[0]
P_n_up_t = result.expect[1:1+n_nuclei]
Sx_t, Sy_t, Sz_t = result.expect[1+n_nuclei:]

# ------------------------
# Plots
# ------------------------
plt.figure()
plt.plot(tgrid*1e6, P_e_up_t, label="Electron")
for k in range(n_nuclei):
    plt.plot(tgrid*1e6, P_n_up_t[k], label=f"Nucleus {k}")
plt.xlabel("time [µs]")
plt.ylabel(r"$P_\uparrow(t)$")
plt.legend()
plt.show()

plt.figure()
plt.plot(tgrid*1e6, Sx_t, label="<Sx>")
plt.plot(tgrid*1e6, Sy_t, label="<Sy>")
plt.plot(tgrid*1e6, Sz_t, label="<Sz>")
plt.xlabel("time [µs]")
plt.ylabel("Spin expectation")
plt.legend()
plt.show()


In [ ]:
import numpy as np
from qutip import *

# Parameters
w = 1.0                # Hamiltonian frequency
H = w * sigmaz() / 2   # Simple 2-level Hamiltonian
psi0 = basis(2, 0)     # Initial state |0>
tlist = np.linspace(0, 10, 200)

# Solve Schrödinger equation
result = sesolve(H, psi0, tlist)

# result.states is a list of state vectors (kets)
psi_t = result.states

print(psi_t[0])        # at t = 0
print(psi_t[50])       # at some later time
print(psi_t)

import matplotlib.pyplot as plt

pop0 = [abs(state[0,0])**2 for state in psi_t]
pop1 = [abs(state[1,0])**2 for state in psi_t]

plt.plot(tlist, pop0, label="|0>")
plt.plot(tlist, pop1, label="|1>")
plt.legend()
plt.xlabel("t")
plt.ylabel("Population")
plt.show()



$(\alpha | \uparrow \rangle_e + \beta | \downarrow \rangle_e) \otimes | 0 \rangle_n \rightarrow | \uparrow \rangle_e \otimes (\alpha | 0 \rangle_n + i \beta | 1 \rangle_n) $

In [ ]:
"""
Full simulation of coherent electron → nuclear state transfer
for a system with 1 electron + 2 nuclei.

Includes:
- alpha/beta initial superposition
- full time-dependent Hamiltonian (ESR + NMR)
- storage of full state kets
- visualisation of all 8 basis-state populations
- 2×4 amplitude matrix heatmap
- fidelity to target state
"""

import numpy as np
import matplotlib.pyplot as plt
import qutip
from qutip import SolverOptions

# ================================================================
# 1. CONFIGURATION
# ================================================================
use_heterogeneous_coupling = True
hetero_dist = "uniform"
n_nuclei = 2  # FIXED to 2 for clarity of the transfer

# ================================================================
# 2. CONSTANTS
# ================================================================
MHz = 1e6
omega_eZ = 0
omega_NZ = 0
a_mean = 2 * np.pi * 0.5 * MHz

omega_1_MW = 2*np.pi*0.05*MHz
omega_MW   = 2*np.pi*5*MHz

omega_1_RF = 2*np.pi*0.02*MHz
omega_RF   = 2*np.pi*1*MHz

# ================================================================
# 3. SPIN OPERATORS
# ================================================================
sx, sy, sz = 0.5*qutip.sigmax(), 0.5*qutip.sigmay(), 0.5*qutip.sigmaz()
sp, sm = 0.5*qutip.sigmap(), 0.5*qutip.sigmam()

# electron operator extended to full space
def e(op):
    return qutip.tensor([op] + [qutip.qeye(2) for _ in range(n_nuclei)])

# nuclear operator acting on k-th nucleus
def n(op, k):
    ops = [qutip.qeye(2)]*(1+n_nuclei)
    ops[k+1] = op
    return qutip.tensor(ops)

# ================================================================
# 4. HYPERFINE COUPLINGS
# ================================================================
if use_heterogeneous_coupling:
    if hetero_dist == "uniform":
        a_list = a_mean * np.random.uniform(0.5, 1.5, n_nuclei)
    else:
        a_list = np.random.normal(a_mean, 0.2*a_mean, n_nuclei)
else:
    a_list = [a_mean]*n_nuclei

# ================================================================
# 5. STATIC HAMILTONIAN
# ================================================================
H_eZ = omega_eZ * e(sz)
H_NZ = omega_NZ * sum(n(sz, k) for k in range(n_nuclei))
H_hf = sum(a_list[k] * (e(sp)*n(sm,k) + e(sm)*n(sp,k) + e(sz)*n(sz,k))
          for k in range(n_nuclei))

# ================================================================
# 6. ESR DRIVE (time-dependent)
# ================================================================
H_esr_x = e(sx)
H_esr_y = e(sy)
H_esr = [
    [omega_1_MW * H_esr_x, "cos(wmw*t)"],
    [-omega_1_MW * H_esr_y, "sin(wmw*t)"]
]

# ================================================================
# 7. NMR DRIVE (time-dependent)
# ================================================================
H_nmr = []
for k in range(n_nuclei):
    H_nmr.append([omega_1_RF * n(sx, k), "cos(wrf*t)"])

# ================================================================
# 8. FULL HAMILTONIAN
# ================================================================
H = [H_eZ + H_NZ + H_hf] + H_esr + H_nmr

# ================================================================
# 9. INITIAL STATE: α|↑> + β|↓> on electron, nuclei in |00>
# ================================================================
alpha = 1/np.sqrt(3)
beta  = np.sqrt(2/3)

up     = qutip.basis(2,0)
down   = qutip.basis(2,1)
zero   = qutip.basis(2,0)
one    = qutip.basis(2,1)

psi_electron = alpha*up + beta*down
psi_nuclear  = qutip.tensor([zero, zero])

psi0 = qutip.tensor(psi_electron, psi_nuclear)

# ================================================================
# 10. TIME EVOLUTION
# ================================================================
tgrid = np.linspace(0, 10e-6, 2000)
args = {"wmw": omega_MW, "wrf": omega_RF}

# We request storage of the full state vectors
result = qutip.sesolve(
    H, psi0, tgrid,
    e_ops=[],                    # no expectation operators needed here
    args=args,
    options=SolverOptions(store_states=True)
)

states_t = result.states

# ================================================================
# 11. TARGET STATE for fidelity calculation
# ================================================================
psi_target = alpha * qutip.tensor(up, zero, zero) \
           + 1j*beta * qutip.tensor(up, one, one)

# ================================================================
# 12. FIDELITY OVER TIME
# ================================================================
F = np.zeros(len(states_t))
for i, state in enumerate(states_t):
    F[i] = np.abs((psi_target.dag() * state)[0,0])**2

plt.figure()
plt.plot(tgrid*1e6, F)
plt.xlabel("time [µs]")
plt.ylabel("Fidelity with target state")
plt.title("Electron → Nuclear Coherent State Transfer")
plt.ylim(0, 1.05)
plt.show()

# ================================================================
# 13. POPULATIONS OF ALL 8 BASIS STATES
# ================================================================
probs = np.zeros((len(states_t), 8))

for ti, state in enumerate(states_t):
    vec = state.full().flatten()
    probs[ti,:] = np.abs(vec)**2

plt.figure(figsize=(10,6))
for k in range(8):
    plt.plot(tgrid*1e6, probs[:,k], label=f"|{k}⟩")
plt.xlabel("time [µs]")
plt.ylabel("Population")
plt.title("Populations of All 8 Basis States")
plt.legend()
plt.show()

# ================================================================
# 14. 2×4 AMPLITUDE MATRIX VISUALIZATION
# ================================================================
def plot_amplitudes_matrix(psi, title="Amplitude Matrix"):
    """
    Reshapes the 8-component state vector into a 2×4 matrix:
    Row 0: electron ↑ amplitudes for nuclear |00>,|01>,|10>,|11>
    Row 1: electron ↓ amplitudes for nuclear |00>,|01>,|10>,|11>
    """
    v = psi.full().flatten()
    M = v.reshape((2,4))

    fig, ax = plt.subplots()
    im = ax.imshow(np.abs(M)**2, interpolation='nearest', aspect='auto')
    plt.colorbar(im, ax=ax)

    ax.set_xticks([0,1,2,3])
    ax.set_xticklabels(["00","01","10","11"])
    ax.set_yticks([0,1])
    ax.set_yticklabels(["↑e","↓e"])
    ax.set_xlabel("Nuclear basis state")
    ax.set_ylabel("Electron state")
    ax.set_title(title)

    plt.show()

# Visualise initial and final states
plot_amplitudes_matrix(states_t[0],   "Initial State Amplitudes")
plot_amplitudes_matrix(states_t[-1],  "Final State Amplitudes")


In [ ]:
# state_transfer_qutip_fixed.py
# ---------------------------------------------------------------------
# Simulation: 1 electron spin + 2 nuclear spins
# Visualisations: full 8-state populations, 2x4 amplitude heatmaps,
#                 fidelity to target transferred state
# Fix: use psi_target.overlap(psi) to compute overlaps (avoids indexing a complex)
# ---------------------------------------------------------------------

import numpy as np
import matplotlib.pyplot as plt
import qutip
from qutip import SolverOptions

# ------------------------
# Parameters
# ------------------------
MHz = 1e6
omega_eZ = 0
omega_NZ = 0
a_mean = 2 * np.pi * 0.5 * MHz

# ESR parameters
omega_1_MW = 2*np.pi*0.05*MHz
omega_MW = 2*np.pi*5*MHz

# NMR parameters
omega_1_RF = 2*np.pi*0.02*MHz
omega_RF = 2*np.pi*1*MHz

n_nuclei = 2

# ------------------------
# Spin-1/2 operators
# ------------------------
sx, sy, sz = 0.5*qutip.sigmax(), 0.5*qutip.sigmay(), 0.5*qutip.sigmaz()
sp, sm = 0.5*qutip.sigmap(), 0.5*qutip.sigmam()

# ------------------------
# Tensor operators in full space (electron + nuclei)
# ------------------------
def e(op):
    """Operator acting on electron, identity on nuclei"""
    return qutip.tensor([op] + [qutip.qeye(2) for _ in range(n_nuclei)])

def n(op, k):
    """Operator acting on nucleus k (0-based), identity elsewhere"""
    ops = [qutip.qeye(2)]*(1+n_nuclei)
    ops[k+1] = op
    return qutip.tensor(ops)

# ------------------------
# Hyperfine couplings (heterogeneous)
# ------------------------
a_list = a_mean * np.random.uniform(0.5, 1.5, n_nuclei)

# ------------------------
# Static Hamiltonian pieces
# ------------------------
H_eZ = omega_eZ * e(sz)
H_NZ = omega_NZ * sum(n(sz, k) for k in range(n_nuclei))
H_hf = sum(a_list[k] * (e(sp)*n(sm,k) + e(sm)*n(sp,k) + e(sz)*n(sz,k))
          for k in range(n_nuclei))

# ------------------------
# Time-dependent driving (ESR + NMR)
# ------------------------
H_esr = [
    [omega_1_MW * e(sx), "cos(wmw*t)"],
    [-omega_1_MW * e(sy), "sin(wmw*t)"]
]

H_nmr = []
for k in range(n_nuclei):
    H_nmr.append([omega_1_RF * n(sx, k), "cos(wrf*t)"])

# ------------------------
# Full Hamiltonian
# ------------------------
H = [H_eZ + H_NZ + H_hf] + H_esr + H_nmr

# ------------------------
# Initial state: (alpha|↑> + beta|↓>) ⊗ |00>
# ------------------------
up = qutip.basis(2,0)
down = qutip.basis(2,1)
zero = qutip.basis(2,0)
one  = qutip.basis(2,1)

# nontrivial superposition (normalized)
alpha = 1/np.sqrt(3)
beta  = np.sqrt(2/3)

psi0 = alpha*qutip.tensor(up, zero, zero) + beta*qutip.tensor(down, zero, zero)

# ------------------------
# Time grid and solver args
# ------------------------
tgrid = np.linspace(0, 10e-6, 1500)
args = {"wmw": omega_MW, "wrf": omega_RF}

# projectors (used as expectation ops, optional)
P_up_e = e(up*up.dag())
P_up_n = [n(up*up.dag(), k) for k in range(n_nuclei)]

# ------------------------
# Solve (store states)
# ------------------------
result = qutip.sesolve(
    H,
    psi0,
    tgrid,
    e_ops=[P_up_e] + P_up_n,      # still collect these expectations if desired
    args=args,
    options=SolverOptions(store_states=True)
)

states_t = result.states  # list of Qobj kets

# ---------------------------------------------------------------------
# VISUALISATION 1: Full 8 basis-state populations over time
# ---------------------------------------------------------------------
probs = np.zeros((len(states_t), 8))
for i, psi in enumerate(states_t):
    vec = psi.full().flatten()
    probs[i,:] = np.abs(vec)**2

plt.figure(figsize=(10,6))
for k in range(8):
    plt.plot(tgrid*1e6, probs[:,k], label=f"|{k:03b}>")
plt.xlabel("time [µs]")
plt.ylabel("Population")
plt.title("Populations of All 8 Basis States")
plt.legend(loc='upper right', fontsize='small')
plt.tight_layout()
plt.show()

# ---------------------------------------------------------------------
# VISUALISATION 2: 2x4 amplitude matrix (electron × nuclei)
# ---------------------------------------------------------------------
def plot_amplitudes_matrix(psi, title="Amplitude Matrix (probabilities)"):
    """
    Display a 2x4 heatmap of probabilities |c_{e,n1n2}|^2:
      rows: electron state (↑, ↓)
      cols: nuclear basis (00, 01, 10, 11)
    psi: Qobj ket (dimension 8)
    """
    vec = psi.full().flatten()
    mat = vec.reshape((2,4))   # shape: (electron=2, nuclei=4)

    fig, ax = plt.subplots(figsize=(6,3))
    im = ax.imshow(np.abs(mat)**2, interpolation='nearest', aspect='auto')
    plt.colorbar(im, ax=ax, label='Population')

    ax.set_xticks([0,1,2,3])
    ax.set_xticklabels(["00","01","10","11"])
    ax.set_yticks([0,1])
    ax.set_yticklabels(["↑e","↓e"])
    ax.set_xlabel("Nuclear basis state (n1 n2)")
    ax.set_ylabel("Electron state")
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

# Plot initial and final amplitude matrices
plot_amplitudes_matrix(states_t[0], title="Initial state amplitudes")
plot_amplitudes_matrix(states_t[-1], title="Final state amplitudes")

# ---------------------------------------------------------------------
# VISUALISATION 3: Fidelity to the target transferred state
#   target: |↑> ⊗ (alpha |00> + i beta |11>)
# ---------------------------------------------------------------------
psi_target = alpha*qutip.tensor(up, zero, zero) + 1j*beta*qutip.tensor(up, one, one)

F = np.zeros(len(states_t))
for i, psi in enumerate(states_t):
    # use overlap() which returns <psi_target|psi> as a complex scalar
    amp = psi_target.overlap(psi)   # complex number
    F[i] = np.abs(amp)**2           # fidelity = |<target|psi>|^2

plt.figure(figsize=(8,4))
plt.plot(tgrid*1e6, F, '-')
plt.xlabel("time [µs]")
plt.ylabel("Fidelity")
plt.title("Fidelity to target transferred state")
plt.ylim(-0.02, 1.05)
plt.grid(True, linestyle=':', alpha=0.5)
plt.tight_layout()
plt.show()

# ---------------------------------------------------------------------
# Optional: print some diagnostics
# ---------------------------------------------------------------------
print("Hyperfine couplings a_list:", a_list)
print("Max fidelity achieved:", F.max())
print("Time of max fidelity (µs):", tgrid[np.argmax(F)]*1e6)

